# TP 4: Apprentissage profond pour les jeux 

### Exercice 1: Initialisation de l'environnement et des structures de données

In [1]:
import gymnasium as gym
import numpy as np

# Initialisation de l'environnement 
env = gym.make("Taxi-v3")

# Nombre d'etats et d'actions 
state_size = env.observation_space.n
action_size = env.action_space.n

In [2]:
# Création de la table de politique (policy_table)
policy_table = np.ones((state_size, action_size)) / action_size

# Création de la table de valeurs (value_table)
value_table = np.zeros(state_size)

# Affichage des premières lignes des tables
print("Premières lignes de policy_table:")
print(policy_table[:5])

print("\nPremières valeurs de value_table:")
print(value_table[:5])


Premières lignes de policy_table:
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]]

Premières valeurs de value_table:
[0. 0. 0. 0. 0.]


### Exercice 2:  Exploration et collecte d'épisodes

In [3]:
# Nombre d'épisodes
num_episodes = 20

for episode in range(num_episodes):
    state, _ = env.reset()  # Réinitialisation de l'environnement
    done = False
    total_reward = 0
    steps = []

    while not done:
        action = env.action_space.sample()  # Sélection d'une action aléatoire
        next_state, reward, done, truncated, info = env.step(action)  # Exécution de l'action
        total_reward += reward
        steps.append((state, action, reward))
        state = next_state  # Mise à jour de l'état

    # Affichage des résultats de l'épisode
    print(f"Épisode {episode + 1}:")
    print(f"  Actions et récompenses: {steps}")
    print(f"  Récompense totale: {total_reward}\n")

# Fermeture de l'environnement
#env.close()


Épisode 1:
  Actions et récompenses: [(42, np.int64(2), -1), (62, np.int64(2), -1), (82, np.int64(0), -1), (182, np.int64(3), -1), (162, np.int64(1), -1), (62, np.int64(1), -1), (62, np.int64(0), -1), (162, np.int64(1), -1), (62, np.int64(2), -1), (82, np.int64(5), -10), (82, np.int64(2), -1), (82, np.int64(0), -1), (182, np.int64(4), -10), (182, np.int64(5), -10), (182, np.int64(4), -10), (182, np.int64(4), -10), (182, np.int64(2), -1), (182, np.int64(5), -10), (182, np.int64(2), -1), (182, np.int64(5), -10), (182, np.int64(2), -1), (182, np.int64(2), -1), (182, np.int64(2), -1), (182, np.int64(3), -1), (162, np.int64(4), -10), (162, np.int64(1), -1), (62, np.int64(1), -1), (62, np.int64(2), -1), (82, np.int64(5), -10), (82, np.int64(4), -10), (82, np.int64(5), -10), (82, np.int64(1), -1), (82, np.int64(0), -1), (182, np.int64(0), -1), (282, np.int64(4), -10), (282, np.int64(5), -10), (282, np.int64(2), -1), (282, np.int64(3), -1), (262, np.int64(5), -10), (262, np.int64(4), -10), (26

### Exercice 3: Mise à jour de la politique avec PPO

In [4]:
gamma = 0.99
lr_policy = 0.1
lr_value = 0.1  # Learning rate pour la mise à jour de la fonction de valeur
clip_epsilon = 0.2

# Exemple de stockage des valeurs d'état
value_table = {}

# Exemple de mise à jour avec des données fictives
policy_table = {}
episode_states = ['state1', 'state2', 'state3']
episode_actions = ['a1', 'a2', 'a3']
episode_rewards = [1, 0, -1]

# Fonction pour calculer les récompenses cumulées (discounted rewards)
def compute_discounted_rewards(rewards, gamma):
    discounted_rewards = np.zeros_like(rewards, dtype=np.float32)
    cumulative_reward = 0
    for t in reversed(range(len(rewards))):
        cumulative_reward = rewards[t] + gamma * cumulative_reward
        discounted_rewards[t] = cumulative_reward
    return discounted_rewards

# Fonction pour calculer l'avantage A_t
def compute_advantage(discounted_rewards, states, value_table):
    advantages = np.zeros_like(discounted_rewards, dtype=np.float32)
    for i, state in enumerate(states):
        value = value_table.get(state, 0)
        advantages[i] = discounted_rewards[i] - value
    return advantages

# Fonction de mise à jour de la politique avec clipping
def update_policy(states, actions, advantages, policy_table):
    for i, state in enumerate(states):
        old_prob = policy_table.get((state, actions[i]), 1.0)
        ratio = old_prob / (old_prob + 1e-8)  # Éviter la division par zéro
        clipped_ratio = np.clip(ratio, 1 - clip_epsilon, 1 + clip_epsilon)
        policy_update = min(ratio * advantages[i], clipped_ratio * advantages[i])
        policy_table[(state, actions[i])] = old_prob + lr_policy * policy_update

# Fonction de mise à jour de la fonction de valeur
def update_value_function(states, discounted_rewards, value_table, lr_value):
    for i, state in enumerate(states):
        old_value = value_table.get(state, 0)
        value_table[state] = old_value + lr_value * (discounted_rewards[i] - old_value)

discounted_rewards = compute_discounted_rewards(episode_rewards, gamma)
advantages = compute_advantage(discounted_rewards, episode_states, value_table)
update_policy(episode_states, episode_actions, advantages, policy_table)
update_value_function(episode_states, discounted_rewards, value_table, lr_value)

print("Policy Table:", policy_table)
print("Value Table:", value_table)


Policy Table: {('state1', 'a1'): np.float64(1.0019899999523838), ('state2', 'a2'): np.float32(0.901), ('state3', 'a3'): np.float32(0.9)}
Value Table: {'state1': np.float32(0.00199), 'state2': np.float32(-0.099), 'state3': np.float32(-0.1)}


### Eexercice 4: 

In [7]:
num_eval_episodes = 20  # Nombre d'épisodes d'évaluation

# Tables de politique et de valeur
policy_table = np.ones((state_size, action_size)) / action_size 
value_table = np.zeros(state_size)  


# Entraînement de l'agent avec PPO
num_episodes = 20
for episode in range(num_episodes):
    state, _ = env.reset()
    episode_states = []
    episode_actions = []
    episode_rewards = []
    done = False
    
    while not done:
        action = np.random.choice(action_size, p=policy_table[state])
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        
        episode_states.append(state)
        episode_actions.append(action)
        episode_rewards.append(reward)
        
        state = next_state
    
    # Calcul des récompenses cumulées
    discounted_rewards = compute_discounted_rewards(episode_rewards, gamma)
    
    for t, state in enumerate(episode_states):
        advantage = discounted_rewards[t] - value_table[state]
        
        # Mise à jour de la valeur de l'état
        value_table[state] += lr_policy * advantage
        
        # Mise à jour de la politique avec clipping
        old_prob = policy_table[state, episode_actions[t]]
        new_prob = old_prob + lr_policy * advantage
        new_prob = np.clip(new_prob, old_prob * (1 - clip_epsilon), old_prob * (1 + clip_epsilon))
        policy_table[state, episode_actions[t]] = new_prob
    
    # Normalisation de la politique pour chaque état visité
    for state in set(episode_states):
        policy_table[state] /= np.sum(policy_table[state])

# Évaluation de l'agent après entraînement
total_rewards = 0
for ep in range(num_eval_episodes):
    state, _ = env.reset()
    done = False
    episode_reward = 0
    
    while not done:
        action = np.argmax(policy_table[state])  # Prendre l'action optimale après entraînement
        state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        episode_reward += reward
    
    total_rewards += episode_reward

# Calcul de la moyenne des récompenses
average_reward = total_rewards / num_eval_episodes
print(f"Score moyen après entraînement : {average_reward}")

# Évaluation avant entraînement (par exemple, avec une politique aléatoire)
total_rewards_before = 0
for ep in range(num_eval_episodes):
    state, _ = env.reset()
    done = False
    episode_reward = 0
    
    while not done:
        action = np.random.choice(action_size, p=policy_table[state])  # Politique aléatoire avant entraînement
        state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        episode_reward += reward
    
    total_rewards_before += episode_reward

# Calcul de la moyenne des récompenses avant l'entraînement
average_reward_before = total_rewards_before / num_eval_episodes
print(f"Score moyen avant entraînement (politique aléatoire) : {average_reward_before}")


Score moyen après entraînement : -469.55
Score moyen avant entraînement (politique aléatoire) : -772.85
